In [1]:
import itertools
from time import sleep
import numpy as np
import os
import pandas as pd
import re
import pathlib
import csv
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.metrics import precision_score

2022-07-24 21:45:48.354326: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-24 21:45:48.354342: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
DATASET_ROOT = os.path.join(os.path.expanduser("~"),
                            'dataSet/audio/agender_distribution/')
VALID_SPLIT = 0.1
SHUFFLE_SEED = 43
BATCH_SIZE = 128
EPOCHS = 100

In [3]:
def map_func(npy_path):
    npy_content = np.load(npy_path)
    return npy_content

In [4]:
train_file_list = pd.read_csv(
    '/home/ferreiraa/Mestrado-PC/github/Conv1D/CNN/file_lists/train_database_normalized.csv')
train_audio_files = train_file_list['file']
train_classes = train_file_list['class']
train_audio_df = pd.DataFrame(train_audio_files)
train_class_df = pd.DataFrame(train_classes)

train_class_labels = list(train_classes.unique())
print(train_class_labels)

audio_paths = []
labels = []

[1, 2, 4, 6, 3, 5, 7]


In [5]:
for label, category in enumerate(train_class_labels):
    print("Processing category {}".format(category,))
    speaker_sample_paths = [os.path.join(DATASET_ROOT, train_audio_files[i])
                            for i in range(len(train_audio_files))
                            if train_classes[i] == category]
    audio_paths += speaker_sample_paths
    labels += [category] * len(speaker_sample_paths)

Processing category 1
Processing category 2
Processing category 4
Processing category 6
Processing category 3
Processing category 5
Processing category 7


In [6]:
for i in range(len(audio_paths)):
    audio_paths[i] = re.sub('.mfc.csv', '.npy', audio_paths[i])

rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(audio_paths)
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(labels)

In [7]:
path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)

2022-07-24 21:46:15.772677: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-24 21:46:15.772698: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-24 21:46:15.772713: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fedora): /proc/driver/nvidia/version does not exist
2022-07-24 21:46:15.772936: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
array_ds = path_ds.map(lambda x: tf.numpy_function(map_func, [x], tf.float64))

In [9]:
label_ds = tf.data.Dataset.from_tensor_slices(labels)

In [10]:
train_ds = tf.data.Dataset.zip((array_ds, label_ds))

In [13]:
list(train_ds.as_numpy_iterator())

[(array([[-19.08524   ,  -6.315624  ,  -1.474195  , ...,   0.5137182 ,
            0.02320643,   0.2472808 ],
         [-16.26493   , -10.40332   ,  -3.897128  , ...,   0.8287333 ,
            0.1031924 ,   0.8747264 ],
         [-15.70372   ,  -8.756201  ,  -2.236906  , ...,   0.5669932 ,
           -0.147231  ,   1.121371  ],
         ...,
         [  4.427859  ,   4.080281  , -15.05049   , ...,   0.443565  ,
            1.004028  ,  -0.1879603 ],
         [  3.368069  ,   0.3517919 , -14.83305   , ...,   0.6968664 ,
            0.9701249 ,  -0.1594324 ],
         [  2.607564  ,  -2.125271  ,  -8.03597   , ...,   0.3250747 ,
            0.3344184 ,  -0.09781663]]),
  2),
 (array([[-11.71981   ,  -9.376075  ,  -2.971919  , ...,  -0.1256654 ,
            0.8698165 ,   0.2662259 ],
         [-15.24199   , -10.91238   ,  -4.877249  , ...,   0.4178837 ,
            1.243966  ,   0.7825798 ],
         [-14.99626   , -11.74381   ,  -5.349494  , ...,   0.9427695 ,
            1.036225  ,   0